# Out-of-Sample Inference (2024-2025)

This notebook demonstrates Phase 2:
1.  **Load Prompts**: Load the optimized agent instructions from Phase 1.
2.  **Inference**: Run the **World Model Inference** (Event-Driven) on future data (2024-2025). This iterates day-by-day, revealing data incrementally to ensure zero look-ahead bias.
3.  **Evaluation**: Assess the performance of the "trained" agent system on unseen data.


In [1]:
import os
import sys
import json
import pandas as pd
from pathlib import Path

# Add project root to path
project_root = Path("../").resolve()
sys.path.insert(0, str(project_root))
sys.path.insert(0, str(project_root / "FinAgents" / "orchestrator_demo"))
sys.path.insert(0, str(project_root / "FinAgents" / "agent_pools"))

from FinAgents.orchestrator_demo.orchestrator import Orchestrator

orchestrator = Orchestrator()
print("✅ Orchestrator Initialized")

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
[7525:MainThread](2025-11-30 04:13:11,996) INFO - qlib.Initialization - [config.py:452] - default_conf: client.
[7525:MainThread](2025-11-30 04:13:11,997) WARNING - qlib.Initialization - [__init__.py:65] - auto_path is False, please make sure None is mounted
[7525:MainThread](2025-11-30 04:13:12,000) INFO - qlib.Initialization - [__init__.py:75] - qlib successfully initialized based on client settings.
[7525:MainThread](2025-11-30 04:13:12,000) INFO - qlib.Initialization - [__init__.py:77] - dat

✅ Qlib system initialized successfully
✅ Qlib components loaded successfully
✅ Orchestrator Initialized


### Step 1: Load Optimized Prompts

In [2]:
prompts_path = "optimized_prompts.json"

if os.path.exists(prompts_path):
    with open(prompts_path, "r") as f:
        optimized_prompts = json.load(f)
    
    # Apply prompts to agents
    orchestrator.alpha_agent.agent.instructions = optimized_prompts.get("Alpha", "")
    orchestrator.risk_agent.agent.instructions = optimized_prompts.get("Risk", "")
    orchestrator.portfolio_agent.agent.instructions = optimized_prompts.get("Portfolio", "")
    
    print("✅ Optimized prompts loaded and applied.")
else:
    print("⚠️ Optimized prompts file not found. Using default instructions.")

✅ Optimized prompts loaded and applied.


### Step 2: Run Out-of-Sample Test (World Model Mode)

We run the backtest for the future period (2024-2025) using **World Model Inference**. This iterates day-by-day, incrementally revealing data to the agent to ensure **zero look-ahead bias**.

In [3]:
symbol = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'META', 'TSLA', 'JPM', 'V', 'WMT']
start_date = "2024-09-01"
end_date = "2025-01-01" # Or today's date

print(f"🚀 Running Out-of-Sample Test for {symbol} ({start_date} to {end_date})...")

# Run Event-Driven World Model Inference (Step-by-Step)
print("🌍 Starting World Model Inference (Day-by-Day Simulation)...")
result = orchestrator.run_inference_rolling_week(symbol, start_date, end_date)

if result and result.get('status') == 'success':
    metrics = result.get('performance_metrics', {})
    print("\n📊 Out-of-Sample Performance Results:")
    print(f"   Total Return: {metrics.get('total_return', 0):.2%}")
    print(f"   Sharpe Ratio: {metrics.get('sharpe_ratio', 0):.2f}")
    print(f"   Max Drawdown: {metrics.get('max_drawdown', 0):.2%}")
else:
    print("❌ Test failed.")

2025-11-30 04:13:12,044 - Orchestrator - INFO - 🚀 Starting World Model Inference (Rolling Week) for ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'META', 'TSLA', 'JPM', 'V', 'WMT'] (2024-09-01 to 2025-01-01)
2025-11-30 04:13:12,045 - Orchestrator - INFO - Fetching data for ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'META', 'TSLA', 'JPM', 'V', 'WMT'] from 2023-09-02 00:00:00 to 2025-01-01 00:00:00


🚀 Running Out-of-Sample Test for ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'META', 'TSLA', 'JPM', 'V', 'WMT'] (2024-09-01 to 2025-01-01)...
🌍 Starting World Model Inference (Day-by-Day Simulation)...


2025-11-30 04:13:14,282 - Orchestrator - INFO - ✅ Successfully fetched real data via yfinance for 10 symbols
2025-11-30 04:13:14,285 - Orchestrator - INFO - 📅 Rolling Step: 2024-09-01 -> 2024-09-08 (Train: 2500, Test: 40)


DEBUG: 🤖 Requesting Alpha Agent LLM...


2025-11-30 04:13:16,831 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


...

2025-11-30 04:15:16,454 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"
2025-11-30 04:15:16,842 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


DEBUG: 🛠️ submit_signals_tool INVOKED


2025-11-30 04:15:19,075 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


DEBUG: LLM finished. Context keys: ['data', 'train_data', 'factors', 'indicators', 'model_type', 'signal_threshold', 'data_processor', 'features', 'raw_predictions', 'result']
🚀 Running simple backtest with value-based accounting
   Period: 2024-09-01 to 2025-01-01
   Capital: $100,000.00

   📊 Portfolio Holdings Snapshot (Sample):
      2024-09-03: Equity=$99,900 (Cash=$49,900), Positions=5
      2024-10-01: Equity=$103,278 (Cash=$61,935), Positions=4
      2024-10-30: Equity=$107,376 (Cash=$4), Positions=10
      2024-11-29: Equity=$114,246 (Cash=$25), Positions=10
      2024-12-31: Equity=$118,163 (Cash=$33), Positions=10

📊 Out-of-Sample Performance Results:
   Total Return: 18.16%
   Sharpe Ratio: 3.10
   Max Drawdown: -4.18%


2025-11-30 04:15:21,964 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"


In [5]:
# Optional: Visualize if available
print("Result Details:", json.dumps(result, indent=2, default=str))

Result Details: {
  "status": "success",
  "performance_metrics": {
    "total_return": 0.18163065305054005,
    "sharpe_ratio": 3.104548069358979,
    "max_drawdown": -0.041753628172510715,
    "volatility": 0.16682099962614894
  },
  "returns_series": "date\n2024-09-03    0.000000\n2024-09-04   -0.004891\n2024-09-05    0.004605\n2024-09-06   -0.014112\n2024-09-09    0.005473\n                ...   \n2024-12-24    0.018637\n2024-12-26   -0.003294\n2024-12-27   -0.016302\n2024-12-30   -0.011851\n2024-12-31   -0.009615\nName: equity, Length: 84, dtype: float64",
  "positions": []
}
